<a href="https://colab.research.google.com/github/jelsinas/Web-Screaping-Jelsi/blob/main/Jelsi_Tugas_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# EKSEKUSI UTAMA - DENGAN TWEET HARVEST
if __name__ == "__main__":
    print("🚀 CNBC SCRAPER + TWEET HARVEST")
    print("🎯 Target: Logam Mulia (Berita + Sosial Media)")
    print("=" * 60)

    # Run testing CNBC dulu
    if test_cnbc_structure():
        print("\n🚀 Testing CNBC OK! Mulai scraping berita...")
        news_filename, news_count = scrape_cnbc_logam_mulia()
        print(f"\n✅ Berita selesai: {news_count} artikel di {news_filename}")

        # Opsional: Jalankan tweet harvest setelah berita
        print("\n🐦 Mulai tweet harvest? (Ya/Tidak)")
        # Untuk auto: Uncomment baris di bawah
        # tweet_filename, tweet_count = harvest_tweets_logam_mulia(max_tweets=200)
        # print(f"✅ Tweet selesai: {tweet_count} tweet di {tweet_filename}")

        # Atau manual: Jalankan fungsi terpisah
    else:
        print("\n❌ Testing CNBC gagal. Cek koneksi.")

🚀 CNBC SCRAPER + TWEET HARVEST
🎯 Target: Logam Mulia (Berita + Sosial Media)
🧪 TESTING STRUKTUR CNBC
✅ Status: 200
📄 Title: Hasil Pencarian  - CNBC Indonesia

🔍 Elemen artikel:
  • div.gsc-webResult: 0
  • div.search-result-item: 0
  • article: 0

🔍 Pagination:

🚀 Testing CNBC OK! Mulai scraping berita...
🚀 MEMULAI SCRAPING CNBC INDONESIA - LOGAM MULIA
🔍 Menganalisis struktur halaman...
  💾 Debug HTML saved to 'debug_cnbc.html'
  ⚠️  Pagination tidak ditemukan, menggunakan default: 5 halaman

💾 Membuat file CSV: cnbc_logam_mulia_20250926_0719.csv

📖 Halaman 1/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=...
    ⚠️  Tidak ada artikel di halaman 1

📖 Halaman 2/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=2...
    ⚠️  Tidak ada artikel di halaman 2

📖 Halaman 3/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=3...
    ⚠️  Tidak ada artikel di halaman 3

📖 Halaman 4/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&p

In [8]:
import tweepy
import csv

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import time
import re
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Fix headers (hapus spasi di User-Agent)
headers = {
    'User -Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Setup session dengan retry untuk handle error transient
session = requests.Session()
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

# Fungsi ambil isi artikel - DIPERBAIKI dengan selector CNBC terkini
def get_article_content(link):
    try:
        print(f"  📖 Mengambil konten: {link[:50]}...")
        res = session.get(link, headers=headers, timeout=15)  # Naikkan timeout
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'lxml')  # Fallback: 'html.parser' jika lxml error

        # Selector CNBC terkini untuk konten (div#detail-tab atau article p)
        content_selectors = [
            'div#detail-tab',
            'div.content-detail',
            'div.detail-text',
            'article p',
            'div[class*="content"] p'
        ]

        article_content = ""
        for selector in content_selectors:
            div_content = soup.select_one(selector)
            if div_content:
                paragraphs = div_content.find_all('p')
                if paragraphs:
                    article_content = ' '.join([p.get_text(strip=True) for p in paragraphs[:10]])  # Limit paragraf
                    break

        # Fallback: Ambil semua p di body
        if not article_content:
            all_paragraphs = soup.find_all('p')
            if len(all_paragraphs) > 3:
                article_content = ' '.join([p.get_text(strip=True) for p in all_paragraphs[:5]])

        return article_content[:800] if article_content else "Konten tidak ditemukan"  # Naikkan limit

    except Exception as e:
        print(f"    ❌ Error artikel: {link[:30]}... | {e}")
        return "Error mengambil konten"

def scrape_cnbc_logam_mulia():
    """Fungsi utama untuk scraping CNBC - FIXED URL dan selector"""
    print("🚀 MEMULAI SCRAPING CNBC INDONESIA - LOGAM MULIA")
    print("=" * 60)

    # URL search CNBC terkini (pakai q= bukan query=; no fromdate/todate)
    base_url = 'https://www.cnbcindonesia.com/search'
    search_params = {'q': 'logam mulia'}  # Keyword utama

    print("🔍 Menganalisis struktur halaman...")

    try:
        # Request halaman pertama dengan session
        res_first = session.get(base_url, params=search_params, headers=headers, timeout=15)
        res_first.raise_for_status()
        soup_first = BeautifulSoup(res_first.text, 'lxml')

        # Debug: Simpan HTML
        with open('debug_cnbc.html', 'w', encoding='utf-8') as f:
            f.write(soup_first.prettify())
        print("  💾 Debug HTML saved to 'debug_cnbc.html'")

        # Cari pagination - FIXED selector untuk CNBC (ul.pager atau div.pagination)
        page_numbers = []
        pagination_selectors = [
            'ul.pager li a',
            'div.pagination a',
            'nav a[aria-label*="page"]',
            'a[href*="page="]'
        ]

        for selector in pagination_selectors:
            links = soup_first.select(selector)
            for a in links:
                text = a.get_text(strip=True)
                if text.isdigit() and int(text) > 1:
                    page_numbers.append(int(text))

                href = a.get('href', '')
                page_match = re.search(r'page=(\d+)', href, re.I)
                if page_match:
                    page_numbers.append(int(page_match.group(1)))

        if page_numbers:
            last_page = max(page_numbers)
            print(f"  📄 Total halaman ditemukan: {last_page}")
        else:
            last_page = 5  # Default
            print(f"  ⚠️  Pagination tidak ditemukan, menggunakan default: {last_page} halaman")

        last_page = min(last_page, 10)  # Batasi testing

    except Exception as e:
        print(f"  ❌ Error analisis: {e}")
        last_page = 3

    # Setup CSV dengan DictWriter untuk robustness
    csv_filename = f'cnbc_logam_mulia_{datetime.datetime.now().strftime("%Y%m%d_%H%M")}.csv'
    print(f"\n💾 Membuat file CSV: {csv_filename}")

    with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
        fieldnames = ['No', 'Title', 'Link', 'Time', 'Content', 'Scraped_Date']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        total_articles = 0

        for page in range(1, last_page + 1):
            print(f"\n📖 Halaman {page}/{last_page}")

            try:
                # FIXED URL pagination: ?q=logam+mulia&page=N
                params = search_params.copy()
                if page > 1:
                    params['page'] = page
                url = base_url
                print(f"  🌐 URL: {base_url}?q=logam+mulia&page={page if page > 1 else ''}...")

                res = session.get(url, params=params, headers=headers, timeout=15)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, 'lxml')

                # FIXED selector artikel: CNBC pakai div.gsc-webResult atau div.search-result
                article_selectors = [
                    'div.gsc-webResult',
                    'div.search-result-item',
                    'div[class*="result"]',
                    'article',
                    'li.result-item'
                ]

                articles = []
                for selector in article_selectors:
                    found = soup.select(selector)
                    if len(found) > 1:  # Minimal 1-2
                        articles = found
                        print(f"    ✅ Selector: {selector} ({len(articles)} items)")
                        break

                if not articles:
                    print(f"    ⚠️  Tidak ada artikel di halaman {page}")
                    continue

                page_count = 0
                for i, art in enumerate(articles[:15]):  # Batasi 15 per halaman
                    try:
                        # FIXED: Cari a_tag dan title
                        a_tag = art.find('a', href=True)
                        if not a_tag:
                            continue

                        title_elem = a_tag.find(['h1', 'h2', 'h3']) or a_tag
                        title = title_elem.get_text(strip=True)[:150]  # Limit title

                        link = a_tag['href']
                        if link.startswith('/'):
                            link = 'https://www.cnbcindonesia.com' + link

                        # Time: Cari span time atau fallback
                        time_elem = art.find('span', class_=re.compile(r'time|date|published'))
                        time_info = time_elem.get_text(strip=True) if time_elem else datetime.datetime.now().strftime('%Y-%m-%d')

                        # Filter keyword
                        if not any(kw in title.lower() for kw in ['logam', 'emas', 'perak', 'mulia']):
                            continue

                        content = get_article_content(link) if link else "Link invalid"

                        # Write row
                        row = {
                            'No': total_articles + 1,
                            'Title': title,
                            'Link': link,
                            'Time': time_info,
                            'Content': content,
                            'Scraped_Date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        }
                        writer.writerow(row)

                        total_articles += 1
                        page_count += 1
                        print(f"    ✅ [{total_articles}] {title[:50]}...")

                    except Exception as e:
                        print(f"    ❌ Error artikel {i}: {e}")
                        continue

                print(f"  📊 Halaman {page}: {page_count} artikel")
                time.sleep(3)  # Delay lebih panjang

            except Exception as e:
                print(f"  ❌ Gagal halaman {page}: {e}")
                continue

    print(f"\n🎉 SCRAPING SELESAI! Total: {total_articles} artikel")
    print(f"💾 File: {csv_filename}")

    # Manual download info (no Colab dependency)
    print(f"📥 Untuk download: Buka folder proyek atau copy {csv_filename}")
    return csv_filename, total_articles

# Fungsi testing - FIXED
def test_cnbc_structure():
    """Test struktur - FIXED URL"""
    print("🧪 TESTING STRUKTUR CNBC")
    print("=" * 50)

    url = 'https://www.cnbcindonesia.com/search?q=logam+mulia'

    try:
        res = session.get(url, headers=headers, timeout=10)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'lxml')

        print(f"✅ Status: {res.status_code}")
        print(f"📄 Title: {soup.title.text if soup.title else 'No title'}")

        # Test selectors
        print("\n🔍 Elemen artikel:")
        selectors = ['div.gsc-webResult', 'div.search-result-item', 'article']
        for sel in selectors:
            count = len(soup.select(sel))
            print(f"  • {sel}: {count}")

        print("\n🔍 Pagination:")
        pag_selectors = ['ul.pager', 'div.pagination']
        for sel in pag_selectors:
            count = len(soup.select(sel))
            if count > 0:
                print(f"  • {sel}: {count} found")

        return True

    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# EKSEKUSI UTAMA
if __name__ == "__main__":
    print("🚀 CNBC SCRAPER FIXED")
    print("🎯 Target: Logam Mulia")
    print("=" * 60)

    # Run testing dulu
    if test_cnbc_structure():
        print("\n🚀 Testing OK! Mulai scraping...")
        filename, count = scrape_cnbc_logam_mulia()
        print(f"\n✅ Selesai: {count} artikel di {filename}")
    else:
        print("\n❌ Testing gagal. Cek koneksi/internet atau update selector.")


🚀 CNBC SCRAPER FIXED
🎯 Target: Logam Mulia
🧪 TESTING STRUKTUR CNBC
✅ Status: 200
📄 Title: Hasil Pencarian  - CNBC Indonesia

🔍 Elemen artikel:
  • div.gsc-webResult: 0
  • div.search-result-item: 0
  • article: 0

🔍 Pagination:

🚀 Testing OK! Mulai scraping...
🚀 MEMULAI SCRAPING CNBC INDONESIA - LOGAM MULIA
🔍 Menganalisis struktur halaman...
  💾 Debug HTML saved to 'debug_cnbc.html'
  ⚠️  Pagination tidak ditemukan, menggunakan default: 5 halaman

💾 Membuat file CSV: cnbc_logam_mulia_20250926_0714.csv

📖 Halaman 1/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=...
    ⚠️  Tidak ada artikel di halaman 1

📖 Halaman 2/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=2...
    ⚠️  Tidak ada artikel di halaman 2

📖 Halaman 3/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=3...
    ⚠️  Tidak ada artikel di halaman 3

📖 Halaman 4/5
  🌐 URL: https://www.cnbcindonesia.com/search?q=logam+mulia&page=4...
    ⚠️  Tidak ada artikel di halaman 